Notebook for running NECom on the stable steady states 
computed using the notebooks "ecoli_model_compute_ss_gne.ipynb"
and "ecoli_model_stability.ipynb".


In [ ]:
import numpy as np
import scipy.io as sio
import scipy.sparse as sparse
import cvxpy as cp
import time
import pickle


In [ ]:
# Set solver to use with cvxpy. See "Choosing a solver" section here for using alternatives 
# to Gurobi: https://www.cvxpy.org/tutorial/advanced/index.html
cp_solver = 'GUROBI'

In [ ]:
# Load data for model.
directory = '../ModelFiles/FourSpecies'

# S contains the stoichiometry matrices R_{k} and R_{k}^{ex} for each species k.
S = sio.loadmat(directory + '/S.mat')['S']
I = sio.loadmat(directory + '/I.mat')['I'][0][0]
J = sio.loadmat(directory + '/J.mat')['J'][0][0]
reaction_lb = sio.loadmat(directory + '/lb.mat')['lb']
reaction_ub = sio.loadmat(directory + '/ub.mat')['ub']

# Indices of reactions and metabolites for each species, needed because 
# cobra groups all metabolites and reactions into a single model.
lumen_reactions_idx = sio.loadmat(directory + '/lumen_reactions_idx.mat')['lumen_reactions_idx'] - 1
lumen_metabolites_idx = sio.loadmat(directory + '/lumen_metabolites_idx.mat')['lumen_metabolites_idx'] - 1
lumen_reaction_names = sio.loadmat(directory + '/lumen_reactions.mat')['lumen_reactions']

Ec1_reactions_idx = sio.loadmat(directory + '/Ec1_reactions_idx.mat')['Ec1_reactions_idx'] - 1
Ec1_reaction_names = sio.loadmat(directory + '/Ec1_reactions.mat')['Ec1_reactions']
Ec1_metabolites_idx = sio.loadmat(directory + '/Ec1_metabolites_idx.mat')['Ec1_metabolites_idx'] - 1
Ec1_biomass_idx = sio.loadmat(directory + '/Ec1_biomass_idx.mat')['Ec1_biomass_idx'][0][0]-1

Ec2_reactions_idx = sio.loadmat(directory + '/Ec2_reactions_idx.mat')['Ec2_reactions_idx'] - 1
Ec2_reaction_names = sio.loadmat(directory + '/Ec2_reactions.mat')['Ec2_reactions']
Ec2_metabolites_idx = sio.loadmat(directory + '/Ec2_metabolites_idx.mat')['Ec2_metabolites_idx'] - 1
Ec2_biomass_idx = sio.loadmat(directory + '/Ec2_biomass_idx.mat')['Ec2_biomass_idx'][0][0]-1

Ec3_reactions_idx = sio.loadmat(directory + '/Ec3_reactions_idx.mat')['Ec3_reactions_idx'] - 1
Ec3_reaction_names = sio.loadmat(directory + '/Ec3_reactions.mat')['Ec3_reactions']
Ec3_metabolites_idx = sio.loadmat(directory + '/Ec3_metabolites_idx.mat')['Ec3_metabolites_idx'] - 1
Ec3_biomass_idx = sio.loadmat(directory + '/Ec3_biomass_idx.mat')['Ec3_biomass_idx'][0][0]-1

Ec4_reactions_idx = sio.loadmat(directory + '/Ec4_reactions_idx.mat')['Ec4_reactions_idx'] - 1
Ec4_reaction_names = sio.loadmat(directory + '/Ec4_reactions.mat')['Ec4_reactions']
Ec4_metabolites_idx = sio.loadmat(directory + '/Ec4_metabolites_idx.mat')['Ec4_metabolites_idx'] - 1
Ec4_biomass_idx = sio.loadmat(directory + '/Ec4_biomass_idx.mat')['Ec4_biomass_idx'][0][0]-1

I1 = len(Ec1_metabolites_idx); I2 = len(Ec2_metabolites_idx); I3 = len(Ec3_metabolites_idx); I4 = len(Ec4_metabolites_idx)
Jl = len(lumen_reactions_idx); J1 = len(Ec1_reactions_idx); J2 = len(Ec2_reactions_idx); J3 = len(Ec3_reactions_idx); J4 = len(Ec4_reactions_idx)


In [ ]:
Ec1_reaction_names = np.array([Ec1_reaction_names[i][0] for i in range(len(Ec1_reaction_names))])
Ec2_reaction_names = np.array([Ec2_reaction_names[i][0] for i in range(len(Ec2_reaction_names))])
Ec3_reaction_names = np.array([Ec3_reaction_names[i][0] for i in range(len(Ec3_reaction_names))])
Ec4_reaction_names = np.array([Ec4_reaction_names[i][0] for i in range(len(Ec4_reaction_names))])
lumen_reaction_names = np.array([lumen_reaction_names[i][0] for i in range(len(lumen_reaction_names))])


In [ ]:
death_rate = np.array([0.50])
stable_steady_states = pickle.load(open("stable_steady_states.p", "rb"))


In [ ]:
[bm1, bm2, bm3, bm4], x1, x2, x3, x4 = stable_steady_states[2]
x1 = bm1 * x1; x2 = bm2 * x2; x3 = bm3 * x3; x4 = bm4 * x4


In [ ]:
# Create vectors that can be dotted with vector of reactions for each species 
# and pull out the biomass reaction.
e1 = sparse.identity(J1 + Jl).tocsr()[:, Ec1_biomass_idx]; e2 = sparse.identity(J2 + Jl).tocsr()[:, Ec2_biomass_idx]
e3 = sparse.identity(J3 + Jl).tocsr()[:, Ec3_biomass_idx]; e4 = sparse.identity(J4 + Jl).tocsr()[:, Ec4_biomass_idx]

# Maximum iterations to find equilibrium.
max_iters = 2000

# Modulate regularization term.
delta_max = 1
delta_min = 1e-3
C = 5e1
B = 6
sigmoid = lambda x : 1 / (1 + np.exp(-x))
k = np.linspace(0, max_iters, max_iters)

delta_vals = delta_min + (delta_max - delta_min) * sigmoid(k/C - B)


In [ ]:
def initial_guess_NECom(bm1, bm2, bm3, bm4, M):
    ''' This is analogous to the function initial_guess 
    in the ecoli_model_compute_ss_gne.ipynb notebook, but 
    with the NECom version of the metabolite exchange constraints. '''
    solution1 = cp.Variable((J1 + Jl, 1))
    solution2 = cp.Variable((J2 + Jl, 1))
    solution3 = cp.Variable((J3 + Jl, 1))
    solution4 = cp.Variable((J4 + Jl, 1))

    beta1 = cp.Variable((Jl, 1))
    delta1 = cp.Variable((Jl, 1), boolean=True)
    beta2 = cp.Variable((Jl, 1))
    delta2 = cp.Variable((Jl, 1), boolean=True)
    beta3 = cp.Variable((Jl, 1))
    delta3 = cp.Variable((Jl, 1), boolean=True)
    beta4 = cp.Variable((Jl, 1))
    delta4 = cp.Variable((Jl, 1), boolean=True)

    objective = 0
    constraints = [
        S[:, np.concatenate([Ec1_reactions_idx, lumen_reactions_idx]).flatten()] @ solution1 == 0,
        solution1[0:J1] >= bm1 * reaction_lb[Ec1_reactions_idx.flatten()],
        solution1[0:J1] <= bm1 * reaction_ub[Ec1_reactions_idx.flatten()],
        S[:, np.concatenate([Ec2_reactions_idx, lumen_reactions_idx]).flatten()] @ solution2 == 0,
        solution2[0:J2] >= bm2 * reaction_lb[Ec2_reactions_idx.flatten()],
        solution2[0:J2] <= bm2 * reaction_ub[Ec2_reactions_idx.flatten()],
        S[:, np.concatenate([Ec3_reactions_idx, lumen_reactions_idx]).flatten()] @ solution3 == 0,
        solution3[0:J3] >= bm3 * reaction_lb[Ec3_reactions_idx.flatten()],
        solution3[0:J3] <= bm3 * reaction_ub[Ec3_reactions_idx.flatten()],
        S[:, np.concatenate([Ec4_reactions_idx, lumen_reactions_idx]).flatten()] @ solution4 == 0,
        solution4[0:J4] >= bm4 * reaction_lb[Ec4_reactions_idx.flatten()],
        solution4[0:J4] <= bm4 * reaction_ub[Ec4_reactions_idx.flatten()],
        solution1[J1:J1+Jl] + solution2[J2:J2+Jl] + solution3[J3:J3+Jl] + solution4[J4:J4+Jl] <= reaction_ub[lumen_reactions_idx.flatten()],
        solution1[J1:J1+Jl] >= beta1,
        beta1 + x2[J2:] + x3[J3:] + x4[J4:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta1 <= 0,
        beta1 + x2[J2:] + x3[J3:] + x4[J4:] + M * (1 - delta1) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta1 + M * delta1 >= 0,
        solution2[J2:J2+Jl] >= beta2,
        beta2 + x1[J1:] + x3[J3:] + x4[J4:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta2 <= 0,
        beta2 + x1[J1:] + x3[J3:] + x4[J4:] + M * (1 - delta2) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta2 + M * delta2 >= 0,
        solution3[J3:J3+Jl] >= beta3,
        beta3 + x2[J2:] + x1[J1:] + x4[J4:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta3 <= 0,
        beta3 + x2[J2:] + x1[J1:] + x4[J4:] + M * (1 - delta3) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta3 + M * delta3 >= 0,
        solution4[J4:J4+Jl] >= beta4,
        beta4 + x2[J2:] + x3[J3:] + x1[J1:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta4 <= 0,
        beta4 + x2[J2:] + x3[J3:] + x1[J1:] + M * (1 - delta4) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta4 + M * delta4 >= 0
    ]
    prob = cp.Problem(cp.Maximize(objective), constraints)
    prob.solve(solver=cp_solver)
    return solution1.value, solution2.value, solution3.value, solution4.value

def compute_ss_NECom(x1_init, x2_init, x3_init, x4_init,
                    bm1, bm2, bm3, bm4, tolerance, M):
    ''' This is analogous to the function compute_steady_state
    in the ecoli_model_compute_ss_gne.ipynb notebook, but 
    with the NECom version of the metabolite exchange constraints. '''
    x1 = x1_init
    x2 = x2_init
    x3 = x3_init
    x4 = x4_init
    current_iter = 0
    current_change = 1e10

    x = np.zeros((J,))
    x[Ec1_reactions_idx.flatten()] = x1[0:J1].flatten()
    x[Ec2_reactions_idx.flatten()] = x2[0:J2].flatten()
    x[Ec3_reactions_idx.flatten()] = x3[0:J3].flatten()
    x[Ec4_reactions_idx.flatten()] = x4[0:J4].flatten()
    x[lumen_reactions_idx.flatten()] = x1[J1:].flatten() + x2[J2:].flatten() + x3[J3:].flatten() + x4[J4:].flatten()

    x_values = np.zeros((max_iters, J)); relative_changes = np.zeros((max_iters-1,1))
    x_values[0,:] = x

    while (current_iter < max_iters) and (current_change > tolerance):
        print('Iteration: ', current_iter)
        start_time = time.time()
        delta = delta_vals[current_iter]

        # Ec1 update.
        #print('Starting Ec1 update')
        solution1 = cp.Variable((J1 + Jl,1))
        beta1 = cp.Variable((Jl, 1))
        delta1 = cp.Variable((Jl, 1), boolean=True)
        objective = e1.T @ solution1 - 0.5 * delta * cp.quad_form(solution1 - x1, sparse.identity(J1+Jl).tocsr())
        constraints = [S[:, np.concatenate([Ec1_reactions_idx, lumen_reactions_idx]).flatten()] @ solution1 == 0,
        solution1[0:J1] >= bm1 * reaction_lb[Ec1_reactions_idx.flatten()],
        solution1[0:J1] <= bm1 * reaction_ub[Ec1_reactions_idx.flatten()],
        solution1[J1:J1+Jl] + x2[J2:] + x3[J3:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()],
        solution1[J1:J1+Jl] >= beta1,
        beta1 + x2[J2:] + x3[J3:] + x4[J4:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta1 <= 0,
        beta1 + x2[J2:] + x3[J3:] + x4[J4:] + M * (1 - delta1) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta1 + M * delta1 >= 0]

        prob1 = cp.Problem(cp.Maximize(objective), constraints)
        try:
            prob1.solve(solver = cp_solver)
        except:
            prob1.solve(solver = cp_solver)
        if solution1.value is None:
            solution1.value = np.zeros((J1 + Jl, 1))
        
        # Ec2 update.
        #print('Starting Ec2 update')
        solution2 = cp.Variable((J2 + Jl, 1))
        beta2 = cp.Variable((Jl, 1))
        delta2 = cp.Variable((Jl, 1), boolean=True)
        objective = e2.T @ solution2 - 0.5 * delta * cp.quad_form(solution2 - x2, sparse.identity(J2+Jl).tocsr())
        constraints = [S[:, np.concatenate([Ec2_reactions_idx, lumen_reactions_idx]).flatten()] @ solution2 == 0,
        solution2[0:J2] >= bm2 * reaction_lb[Ec2_reactions_idx.flatten()],
        solution2[0:J2] <= bm2 * reaction_ub[Ec2_reactions_idx.flatten()],
        solution2[J2:J2+Jl] + x1[J1:] + x3[J3:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()],
        solution2[J2:J2+Jl] >= beta2,
        beta2 + x1[J1:] + x3[J3:] + x4[J4:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta2 <= 0,
        beta2 + x1[J1:] + x3[J3:] + x4[J4:] + M * (1 - delta2) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta2 + M * delta2 >= 0]

        prob2 = cp.Problem(cp.Maximize(objective), constraints)
        try:
            prob2.solve(solver = cp_solver)
        except:
            prob2.solve(solver = cp_solver)
        if solution2.value is None:
            solution2.value = np.zeros((J2 + Jl, 1))
        
        # Ec3 update.
        #print('Starting Ec3 update')
        solution3 = cp.Variable((J3 + Jl, 1))
        beta3 = cp.Variable((Jl, 1))
        delta3 = cp.Variable((Jl, 1), boolean=True)
        objective = e3.T @ solution3 - 0.5 * delta * cp.quad_form(solution3 - x3, sparse.identity(J3+Jl).tocsr())
        constraints = [S[:, np.concatenate([Ec3_reactions_idx, lumen_reactions_idx]).flatten()] @ solution3 == 0,
        solution3[0:J3] >= bm3 * reaction_lb[Ec3_reactions_idx.flatten()],
        solution3[0:J3] <= bm3 * reaction_ub[Ec3_reactions_idx.flatten()],
        solution3[J3:J3+Jl] + x2[J2:] + x1[J1:] + x4[J4:] <= reaction_ub[lumen_reactions_idx.flatten()],
        solution3[J3:J3+Jl] >= beta3,
        beta3 + x2[J2:] + x1[J1:] + x4[J4:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta3 <= 0,
        beta3 + x2[J2:] + x1[J1:] + x4[J4:] + M * (1 - delta3) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta3 + M * delta3 >= 0]

        prob3 = cp.Problem(cp.Maximize(objective), constraints)
        try:
            prob3.solve(solver = cp_solver)
        except:
            prob3.solve(solver = cp_solver)
        if solution3.value is None:
            solution3.value = np.zeros((J3 + Jl, 1))
        
        # Ec4 update.
        #print('Starting Ec4 update')
        solution4 = cp.Variable((J4 + Jl, 1))
        beta4 = cp.Variable((Jl, 1))
        delta4 = cp.Variable((Jl, 1), boolean=True)
        objective = e4.T @ solution4 - 0.5 * delta * cp.quad_form(solution4 - x4, sparse.identity(J4+Jl).tocsr())
        constraints = [S[:, np.concatenate([Ec4_reactions_idx, lumen_reactions_idx]).flatten()] @ solution4 == 0,
        solution4[0:J4] >= bm4 * reaction_lb[Ec4_reactions_idx.flatten()],
        solution4[0:J4] <= bm4 * reaction_ub[Ec4_reactions_idx.flatten()],
        solution4[J4:J4+Jl] + x2[J2:] + x3[J3:] + x1[J1:] <= reaction_ub[lumen_reactions_idx.flatten()],
        solution4[J4:J4+Jl] >= beta4,
        beta4 + x2[J2:] + x3[J3:] + x1[J1:] <= reaction_lb[lumen_reactions_idx.flatten()],
        beta4 <= 0,
        beta4 + x2[J2:] + x3[J3:] + x1[J1:] + M * (1 - delta4) >= reaction_lb[lumen_reactions_idx.flatten()],
        beta4 + M * delta4 >= 0]

        prob4 = cp.Problem(cp.Maximize(objective), constraints)
        try:
            prob4.solve(solver = cp_solver)
        except:
            prob4.solve(solver = cp_solver)
        if solution4.value is None:
            solution4.value = np.zeros((J4 + Jl, 1))
        
        # Update fluxes and biomasses.
        x1 = x1 + (1 / (2 + np.sqrt(current_iter))) * (solution1.value[0:J1+Jl] - x1)
        x2 = x2 + (1 / (2 + np.sqrt(current_iter))) * (solution2.value[0:J2+Jl] - x2)
        x3 = x3 + (1 / (2 + np.sqrt(current_iter))) * (solution3.value[0:J3+Jl] - x3)
        x4 = x4 + (1 / (2 + np.sqrt(current_iter))) * (solution4.value[0:J4+Jl] - x4)
        
        x[Ec1_reactions_idx.flatten()] = x1[0:J1].flatten()
        x[Ec2_reactions_idx.flatten()] = x2[0:J2].flatten()
        x[Ec3_reactions_idx.flatten()] = x3[0:J3].flatten()
        x[Ec4_reactions_idx.flatten()] = x4[0:J4].flatten()
        x[lumen_reactions_idx.flatten()] = x1[J1:].flatten() + x2[J2:].flatten() + x3[J3:].flatten() + x4[J4:].flatten()
        x_values[current_iter,:] = x.flatten()

        if current_iter > 0:
            i = current_iter
            #print('Relative change in fluxes: ', np.linalg.norm((x_values[i,:] - x_values[i-1,:])) / (np.linalg.norm(x_values[i-1,:]) + 1e-6))
            relative_changes[i-1] = np.linalg.norm((x_values[i,:] - x_values[i-1,:])) / (np.linalg.norm(x_values[i-1,:]) + 1e-6)
            current_change = relative_changes[i-1]

        #print('Ec1 unweighted biomass reaction rate: ', e1.T.dot(x1) / bm1)
        #print('Ec2 unweighted biomass reaction rate: ', e2.T.dot(x2) / bm2)
        #print('Ec3 unweighted biomass reaction rate: ', e3.T.dot(x3) / bm3)
        #print('Ec4 unweighted biomass reaction rate: ', e4.T.dot(x4) / bm4)
            
        current_iter = current_iter + 1
        
        #print('Time for Iteration: ', time.time() - start_time, ' seconds')
        #print('\n')
        #print('\n')

    return x1, x2, x3, x4


In [ ]:
sample = np.random.uniform(0,1,size=(1000,4))
sample = sample / np.sum(sample, axis = 1)[:,None]


In [ ]:
NECom_steady_states = []
for i in range(sample.shape[0]):
    print('i: ', i)
    target_bm = sample[i,:]
    try:
        x1_init, x2_init, x3_init, x4_init = initial_guess_NECom(bm1, bm2, bm3, bm4, M)
    except:
        continue
    if (x1 is None) or (x2 is None) or (x3 is None) or (x4 is None):
        continue
    M = 1e6
    try:
        x1_NECom, x2_NECom, x3_NECom, x4_NECom = compute_ss_NECom(x1_init, x2_init, x3_init, x4_init,
                                                                  bm1, bm2, bm3, bm4, 1e-7, M)
        NECom_steady_states.append((target_bm, x1_NECom, x2_NECom, x3_NECom, x4_NECom))
    except:
        continue
    